## Goal

We're going to build an agent which serves as a coding assistant. It'll have the design pattern knowledge provided via vector store and also will be able to write and review the code. The inputs to the agent are:

- Tools: Descriptions of available tools
- User input: High-level objective
- Any (action, tool output) pairs previously executed in order to achieve the user input

In this example, we will use OpenAI Function Calling to create this agent. **This is generally the most reliable way to create agents.**

Per https://platform.openai.com/docs/guides/function-calling
> In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.


In [10]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Tools
Tools are functions that an agent can invoke. There are two important design considerations around tools:

- Giving the agent access to the right tools
- Describing the tools in a way that is most helpful to the agent

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import tool
from langchain.vectorstores import Chroma

# Design patterns tool

loader = CSVLoader(file_path='../data/design_patterns.csv', source_column='content', metadata_columns=['page_title'])
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(split_docs, embedding=embeddings)

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
vs_retriever_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
vector_store_retriever = db.as_retriever(search_kwargs={"k": 3})

vs_retriever_chain = {'question': RunnablePassthrough(), 'context': vector_store_retriever} | QA_CHAIN_PROMPT | vs_retriever_llm | StrOutputParser()

class DesignPatternsInput(BaseModel):
    design_pattern: str = Field(description="name of the design pattern to learn about")
    
@tool("design_patterns_knowledge", args_schema=DesignPatternsInput)
def design_patterns_knowledge(design_pattern: str) -> str:
    """Source of design patterns knowledge. Use it to gain knowledge about certain design patterns."""
    return vs_retriever_chain.invoke(design_pattern)

In [3]:
from langchain.pydantic_v1 import BaseModel, Field

# Code review tool

review_code_prompt = """Write code review of the given code. Consider the following aspects:
- Are there any design patterns applicable?
- Is the code readable?
- Does the code follow the design principles like KISS, SOLID, etc?

Code: {code}

Your review: """

REVIEW_CODE_TEMPLATE = PromptTemplate(
    input_variables=["code"],
    template=review_code_prompt,
)
review_code_llm = ChatOpenAI(temperature=0, model_name="gpt-4")

review_code_chain = REVIEW_CODE_TEMPLATE | review_code_llm | StrOutputParser()

class ReviewCodeInput(BaseModel):
    code: str = Field(description="code to review")

@tool("review_code", args_schema=ReviewCodeInput)
def review_code(code: str) -> str:
    """Use this tool to perform code review."""
    return review_code_chain.invoke({'code': code})

In [4]:
from langchain.pydantic_v1 import BaseModel, Field

# Code writing tool

write_code_prompt = """Write python code based on user description. Try to add descriptive comments to explain what the code does.

Return only python code in Markdown format, e.g.:

```python
....
```

Code description: {description}
Code to refactor: {code_to_refactor}
"""

write_code_template = PromptTemplate(
    input_variables=["description", "code_to_refactor"],
    template=write_code_prompt,
)

def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

write_code_llm = ChatOpenAI(temperature=0, model_name="gpt-4")

write_code_chain = write_code_template | write_code_llm | StrOutputParser() | _sanitize_output

class WriteCodeInput(BaseModel):
    description: str = Field(description="code description to generate")
    code_to_refactor: str = Field(description="optional code to refactor", default="")

@tool("write_code", args_schema=WriteCodeInput)
def write_code(description: str, code_to_refactor: str = "") -> str:
    """Generate python code based on description and optionally, the existing code to refactor."""
    return write_code_chain.invoke({'description': description, 'code_to_refactor': code_to_refactor})

## Agent Executor

The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.

While this may seem simple, there are several complexities this runtime handles for you, including:

- Handling cases where the agent selects a non-existent tool
- Handling cases where the tool errors
- Handling cases where the agent produces output that cannot be parsed into a tool invocation
- Logging and observability at all levels (agent decisions, tool calls) to stdout and/or to LangSmith

How does the agent know what tools it can use? In this case we're relying on OpenAI function calling LLMs, which take functions as a separate argument and have been specifically trained to know when to invoke those functions.


In [11]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

tools = [design_patterns_knowledge, write_code, review_code]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a coding assistant, helping user to write, review and refactor code.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),  # Prompt template will assume this message is already in place
    ]
)

agent_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
# pass constant arguments to a Runnable within a Runnable sequence without them being part of the preceding Runnable's output or the user input
llm_with_tools = agent_llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

# Inputs: user input, agent scratchpad
# user input: high-level objective
# agent scratchpad: list of messages containing previous agent tool invocations
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),  # Convert (AgentAction, tool output) tuples into FunctionMessages.
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input": """
Review the following code, apply the review suggestions to refactor the code and return me the refactored code.

```python
class Square:
    def __init__(self, side_length):
        self.side_length = side_length

    def area(self):
        return self.side_length * self.side_length

    def perimeter(self):
        return 4 * self.side_length

class Circle:
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

    def circumference(self):
        return 2 * 3.14 * self.radius

def calculate_area(objects):
    for obj in objects:
        if isinstance(obj, Square):
            print("Area of square:", obj.area())
        elif isinstance(obj, Circle):
            print("Area of circle:", obj.area())

def calculate_perimeter(objects):
    for obj in objects:
        if isinstance(obj, Square):
            print("Perimeter of square:", obj.perimeter())
        elif isinstance(obj, Circle):
            print("Circumference of circle:", obj.circumference())

# Usage
shapes = [Square(5), Circle(3)]
calculate_area(shapes)
calculate_perimeter(shapes)
```
"""})



> Entering new AgentExecutor chain...

Invoking: `review_code` with `{'code': 'class Square:\n    def __init__(self, side_length):\n        self.side_length = side_length\n\n    def area(self):\n        return self.side_length * self.side_length\n\n    def perimeter(self):\n        return 4 * self.side_length\n\nclass Circle:\n    def __init__(self, radius):\n        self.radius = radius\n\n    def area(self):\n        return 3.14 * self.radius * self.radius\n\n    def circumference(self):\n        return 2 * 3.14 * self.radius\n\ndef calculate_area(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Area of square:", obj.area())\n        elif isinstance(obj, Circle):\n            print("Area of circle:", obj.area())\n\ndef calculate_perimeter(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Perimeter of square:", obj.perimeter())\n        elif isinstance(obj, Circle):\n            print("Circumference

{'input': '\nReview the following code, apply the review suggestions to refactor the code and return me the refactored code.\n\n```python\nclass Square:\n    def __init__(self, side_length):\n        self.side_length = side_length\n\n    def area(self):\n        return self.side_length * self.side_length\n\n    def perimeter(self):\n        return 4 * self.side_length\n\nclass Circle:\n    def __init__(self, radius):\n        self.radius = radius\n\n    def area(self):\n        return 3.14 * self.radius * self.radius\n\n    def circumference(self):\n        return 2 * 3.14 * self.radius\n\ndef calculate_area(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Area of square:", obj.area())\n        elif isinstance(obj, Circle):\n            print("Area of circle:", obj.area())\n\ndef calculate_perimeter(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Perimeter of square:", obj.perimeter())\n        elif 